# Arrows And Barbs Maps Creations From Winds data (u, v, extract from NetCDF file of Era5)



## Prerequisites:

Python (This script was tested using Python 3.11.1) </br>
Free packages: </br>
Folium version ...</br>
Pandas version ...</br>
(Complete at the end)

In [ ]:
import sys

restart = False


try:
    # xarray is used to read NetCDF file
    import xarray as xr
    print("xarray="+str(xr.__version__))
except ImportError:
    print ('xarray is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install xarray
    restart = True

try:
    # matplotlib is used to do graph, arrows, barbs
    import matplotlib
    print("matplotlib="+str(matplotlib.__version__))
    
except ImportError:
    print ('matplotlib is not installed, installing it now. The kernel will restart.')
    !{sys.executable} -m pip install matplotlib
    restart = True      

try:
    # matplotlib is used to do graph, arrows, barbs
    import matplotlib.pyplot as plt    
except ImportError:
    print ('matplotlib is not installed, installing it now. The kernel will restart.')
    !{sys.executable} -m pip install matplotlib
    restart = True    

try:
    # pandas is used to read NetCDF file
    import pandas as pd
    print("pandas="+str(pd.__version__))
except ImportError:
    print ('pandas is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install pandas
    restart = True  
    
# Cartopy is used for cartography data. 
# We check if all needed elements are properly installed
try:
    import cartopy
    print("cartopy="+str(cartopy.__version__))
except ImportError:
    print ('cartopy is not installed, installing it now. The kernel will restart. ')
    !conda install -y --prefix {sys.prefix} cartopy  
    restart = True    

try:  
    import cartopy.crs as ccrs
except ImportError:
    print ('cartopy.crs is not installed, installing it now. The kernel will restart. ')
    !conda install --yes --prefix {sys.prefix} cartopy.crs
    restart = True      
    
    
try:
    import cartopy.feature
except ImportError:
    print ('cartopy.feature is not installed, installing it now. The kernel will restart. ')
    !conda install -y --prefix {sys.prefix} cartopy.feature  
    restart = True 
    
if (restart)   :
    print ('Library installation is finished. Please rerun this script at your convenience')
    sys.exit(1)
    

  

***

# Wind vectors and barbs in a map
***

Velocity requires both:

    U the zonal velocity, i.e. the component of the horizontal wind towards east;
    V is the meridional velocity, i.e. the component of the horizontal wind towards north

The meteorological convention for winds is that U component is positive for a west to east flow (eastward wind) and the V component is positive for south to north flow (northward wind).

The wind data we will be using for this episode is from Copernicus Climate Data Store and is freely available (but registration is required). Data corresponds to ECMWF re-analysis 5 (ERA5) daily data for 25th December 2018 at 12 UTC.



The history of netCDF back to CDF library (common data format) developed by NASA for storage of numerical multidimensional data which is a self-describing, platform-independent database. The idea about netCDF4 is not any different than CDF or HDF5 (we will come to that later) but more user friendly and thus easy for research and advanced applications. 



If you wanted wind barbs, then you would use barbs()

We pass ccrs.PlateCarree() because it seems like your data are in lon/lat space. This implies that your wind components are earth relative in this case. When calling quiver/barbs, cartopy assumes that your coordinates (i.e. x/y or lon/lat) are in the same coordinate system as your vector components (i.e. u/v).



## 1. Data imported from NetCDF

In [ ]:
ncw = xr.open_dataset('uv_ERA5_25122018_small.nc') #change here the name of netcdf file here



# The simple file putted as example here (uv_ERA5_25122018_small.nc) has only one time = 2018-12-25T12:00:00 

##--> If you have a file with different times, used: 
# uvEra5 = xr.open_dataset('dataWithDifferentTime.nc')
## And then select a time (the first january 2000 at 6:00, check the way the time is described in your netcdf file) with: 
# ncw = uvEra5.sel(time='2000-01-01T06')





# Some tips to manipulate netCDF with xarray

display(ncw)

#print(ncw.data_vars)

#print(ncw['time'])

# Storing data:
u = ncw['u10'].dropna('lat').to_dataframe()
v = ncw['v10'].dropna('lat').to_dataframe()


lon = ncw['lon']
lat = ncw['lat']

display(lon)
display(lat)

speed = ((ncw['u10']**2) + ncw['v10']**2)**0.5
display(speed)









***

## Quick Plot of data 
***

In [ ]:



# Quick contour plot of the data 
fig = plt.figure(figsize=(15,15))
plt.contourf(ncw['u10'][:,:])
plt.title("u", fontsize=20)
plt.colorbar()

fig = plt.figure(figsize=(15,15))
plt.contourf(ncw['v10'][:,:])
plt.title("v", fontsize=20)
plt.colorbar()

fig = plt.figure(figsize=(15,15))
plt.contourf(speed[:,:])
plt.title("speed", fontsize=20)
plt.colorbar()






In [ ]:
# ContourPlot with map



from cartopy.feature import NaturalEarthFeature


cbar_kwargs = {
    'orientation':'horizontal',
    'fraction': 0.045,
    'pad': 0.01,
    'extend':'neither'
}


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90])
ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

xr.plot.imshow(speed[:,:], ax=ax, add_labels=False,
                   vmin=0, vmax=24, cmap='Reds', 
                   cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.title("speed", fontsize=20)


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90])
ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

xr.plot.imshow(ncw['u10'][:,:], ax=ax, add_labels=False,
                   vmin=-20, vmax=25, cmap='jet', 
                   cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.title("u", fontsize=20)



fig = plt.figure(figsize=(20,20))

ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())

ax.set_extent([-180, 180, -90, 90])

#Adding contour of continent
ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

xr.plot.imshow(ncw['v10'][:,:], ax=ax, add_labels=False,
                   vmin=-20, vmax=25, cmap='jet', 
                   cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.title("v", fontsize=20)



#plt.contourf(ncw['u10'][:,:])


In [ ]:
# ContourPlot with map and winds arrows
# -- Add arrows to show the wind vectors --


from cartopy.feature import NaturalEarthFeature


cbar_kwargs = {
    'orientation':'horizontal',
    'fraction': 0.045,
    'pad': 0.01,
    'extend':'neither'
}


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90])
ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

xr.plot.imshow(speed[:,:], ax=ax, add_labels=False,
                   vmin=0, vmax=24, cmap='Blues', 
                   cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.title("Wind speeds", fontsize=20)


x = ncw['u10'].lon.values
y = ncw['u10'].lat.values
u = ncw['u10'].values
v = ncw['v10'].values



#Here we add the arrow
Q = plt.quiver( x, y, u, v,
               pivot='middle',  #Arrows in the middle of cells
               transform=ccrs.PlateCarree(), 
               width=0.0005   #Change this value to adjust the arrow size depending on the scale of your map
               )



### plot quiver key
qk = plt.quiverkey(Q, 
                   .980, 1.01,                        # x,y label position in % of map size
                   15, str(15)+' '+ncw['u10'].units,  # choose units + update string
                   labelpos='N',                      # add label to the right
                   coordinates='axes'
                   )

### Save the picture in a picture file
plt.savefig("arrowspeedBlack.png", bbox_inches='tight', dpi=150) #Increase dpi for a better png resolution. Other picture format exist




## We can do the same with barbs:

https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/barb_demo.html?highlight=plt%20barbs

In [ ]:
# Here we add the arrows
# ContourPlot with map and winds arrows
# -- Add arrows to show the wind vectors --


from cartopy.feature import NaturalEarthFeature


cbar_kwargs = {
    'orientation':'horizontal',
    'fraction': 0.045,
    'pad': 0.01,
    'extend':'neither'
}


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90])
ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

xr.plot.imshow(speed[:,:], ax=ax, add_labels=False,
                   vmin=0, vmax=24, cmap='Reds', 
                   cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.title("Wind speed with barbs", fontsize=20)


x = ncw['u10'].lon.values
y = ncw['u10'].lat.values
u = ncw['u10'].values
v = ncw['v10'].values


#Here we add the barbs

Q = plt.barbs( x, y, u, v, 
               pivot='middle', 
               transform=ccrs.PlateCarree(), 
               length=4, linewidth=0.5  #Change this value to adjust the arrow size depending on the scale of your map
               )

# Save data in a picture, here png format. Filename .png for image, .pdf for pdf format. File location can also be specified here.
plt.savefig("barbsSpeedBlack.png", bbox_inches='tight', dpi=300) #Increase dpi for a better png resolution. Other picture format exist


